<a href="https://colab.research.google.com/github/ronicostaf/Proj_IA_Sustentabilidade/blob/main/Proj_IA_Sustentabilidade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

App Rede EcoDrone alpha 0.1

    Rede: Colaboração entre alunos, escolas e fiscais dos munícipios parceiros.
    Eco: Refere-se diretamente à questão ambiental e à preservação.
    Drone: Usa a tecnologia de drones para mapear as áreas verdes e forncer as imagens para a IA

Configurações do clound do Google e parâmetros do modelo

In [120]:
!pip install -q -U google-generativeai

In [121]:
# Import the Python SDK
import google.generativeai as genai
import pandas as pd
import numpy as np
import cv2
import pathlib
import textwrap

import google.generativeai as genai

# INFORMANDO A CHAVE SECRETA
from google.colab import userdata
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

In [122]:
generate_config = {
    'candidate_count': 1,
    'temperature': 0.5
}

# configurações de segurança

safety_settings = {
    'harassment': 'block_none',
    'hate': 'block_none',
    'sexual': 'block_none',
    'dangerous': 'block_none'
}

# inicializando o modelo
model = genai.GenerativeModel(model_name='gemini-1.0-pro',
                              generation_config=generate_config,
                              safety_settings=safety_settings)

Código para melhoria da visualização

In [123]:

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


O trecho de código a seguir analisa a quantidade de verde em uma imagem. O usuário passa o caminho das imagens a serem analizadas e a função vai calcular a porcentagem de pixels verdes que cada imagem contêm.

In [124]:
# analisa a imagem para calcular área verde
def analisar_imagem(caminho_imagem):
  imagem = cv2.imread(caminho_imagem)

  # checando a existência da imagem
  if len(imagem) == 0:
    print('\nFalha no carregamento da imagem!')
  else:
    hsv = cv2.cvtColor(imagem, cv2.COLOR_BGR2HSV)

  # Defina a faixa de cor verde em HSV
  verde_inferior = np.array([35, 50, 50])
  verde_superior = np.array([85, 255, 255])

  # Crie uma máscara para isolar a cor verde
  mascara = cv2.inRange(hsv, verde_inferior, verde_superior)

  # Calcule a porcentagem de pixels verdes
  total_pixels = imagem.shape[0] * imagem.shape[1]
  pixels_verdes = cv2.countNonZero(mascara)
  return (pixels_verdes / total_pixels) * 100

# Obter informações do usuário
prompt = input('Deseja continuar [S ou N]: ')
while prompt != 'N':
  imagem1 = input("Insira o caminho para a primeira imagem: ")
  imagem2 = input("Insira o caminho para a segunda imagem: ")
  bioma = input("Qual o bioma das imagens? [1] Mata Atlântica | [2] Amazônia: ")
  local = input("Qual o local das imagens? [Município/área verde]: ")
  prompt = 'N'

# Observações:
# A detecção de verde ainda é uma simplificação, e fatores como iluminação e sombras podem influenciar a precisão.
# O ajuste fino da faixa HSV para "verde" pode ser necessário dependendo das imagens.
# caminhos das imagens de teste, em ordem:
# imagem1 = '/teste1.jpg'
# imagem2 = '/teste2.jpg'

Deseja continuar [S ou N]: s
Insira o caminho para a primeira imagem: /teste1.jpg
Insira o caminho para a segunda imagem: /teste2.jpg
Qual o bioma das imagens? [1] Mata Atlântica | [2] Amazônia: 1
Qual o local das imagens? [Município/área verde]: Petrópolis/Serra dos Órgãos


Após a coleta das imagens, se calcula a perda vegetativa do local fotografado, por meio da criação de duas variáveis, onde será calculado posteriormente a diferença entre a situação "antes" e a situação "depois".

In [126]:
# Analisar as imagens e calcular a perda vegetativa
cobertura1 = analisar_imagem(imagem1)
cobertura2 = analisar_imagem(imagem2)
perda = cobertura1 - cobertura2
# corrreção caso a ordem das imagens seja invertida
if perda < 0:
  perda = perda * (-1)


O aplicativo mostra a perda vegetativa e possíveis causas de acordo com o bioma inserido pelo usuário.

In [127]:
# Imprimir resultados
print("\nResultados da análise:")
print(f"Imagem 1: {cobertura1:.2f}% de cobertura vegetal")
print(f"Imagem 2: {cobertura2:.2f}% de cobertura vegetal")
print(f"Perda...: {perda:.2f}% de cobertura vegetal")


Resultados da análise:
Imagem 1: 41.02% de cobertura vegetal
Imagem 2: 22.34% de cobertura vegetal
Perda...: 18.68% de cobertura vegetal


De acordo com pesquisas do GEMINI existem causas dierenciadas para cada tipo de bioma, o que pode direcionar melhor a fiscalização.

In [128]:
# Possíveis causas (simplificado - requer lógica mais complexa)
if bioma.lower() == '1':
  print("\nPossíveis causas (Mata Atlântica):")
  print("- Exploração madeireira ilegal")
  print("- Agricultura (plantação de monoculturas)")
  print("- Expansão urbana ou construção de infraestrutura")
  print("- Queimadas")
elif bioma.lower() == '2':
  print("\nPossíveis causas (Floresta Amazônica):")
  print("- Grilagem de terras e especulação imobiliária")
  print("- Agropecuária extensiva (pecuária e soja)")
  print("- Mineração")
  print("- Projetos de infraestrutura (hidrelétricas, estradas)")
else:
  print("\nBioma não reconhecido para sugestão de causas.")


Possíveis causas (Mata Atlântica):
- Exploração madeireira ilegal
- Agricultura (plantação de monoculturas)
- Expansão urbana ou construção de infraestrutura
- Queimadas


Buscando ajuda para a IA em uma solução mais robusta para o problema

In [130]:
# resposta da IA

response = model.generate_content("Baseado nas possíveis causas de perda vegetativa do bioma citado, \n"
"cujas imagens você analisou e também usando os dados públicos do local informado, em órgãos como \n"
"IBAMA e Secretaria de Meio Ambiente do Estado e do Município informado, quais sugestões você poderia dar, \n"
"para contribuir com a criação de soluções para mitigar a diminuição da área verde especificada?")
print(response.text)


**Sugestões para Mitigar a Perda Vegetativa do Bioma**

**Com base nas causas identificadas nas imagens e dados públicos:**

* **Desmatamento ilegal:**
    * Fortalecer a fiscalização e aplicar penalidades rigorosas aos infratores.
    * Criar programas de monitoramento para detectar e prevenir o desmatamento ilegal.

* **Exploração madeireira insustentável:**
    * Implementar práticas de manejo florestal sustentável para garantir a extração responsável de madeira.
    * Promover o uso de madeira certificada de fontes sustentáveis.

* **Queimadas:**
    * Criar campanhas de conscientização sobre os riscos das queimadas e as consequências legais.
    * Melhorar a capacidade de resposta a incêndios florestais e fornecer treinamento para bombeiros.

* **Expansão urbana e agrícola:**
    * Promover o planejamento urbano sustentável que priorize a proteção das áreas verdes.
    * Apoiar práticas agrícolas sustentáveis que minimizem o desmatamento e a degradação ambiental.

**Outras sugestõ